In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from pprint import pprint as pp
import csv
from pathlib import Path
import seaborn as sns
from itertools import product
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline 

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.metrics import homogeneity_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import MiniBatchKMeans, DBSCAN
import fasttext

import gensim
from gensim import corpora

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

In [4]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_predict

In [5]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

In [6]:
pd.set_option('display.max_columns', None)

### Retrieving the data

#### DF 8

In [7]:
df8 = pd.read_csv('df8.csv')

In [11]:
# Drop the 'content' column from df8
df8.drop(columns=['content'], inplace=True)

In [12]:
df8.head(2)

,Unnamed: 0,id,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,pymnt_plan,purpose,title,addr_state,earliest_cr_line,last_pymnt_d,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,acc_now_delinq,chargeoff_within_12_mths,tax_liens,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,zip_code,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,delinq_amnt,pub_rec_bankruptcies,target
0,0,1077501,36months,B,B2,Unknown,10,RENT,Verified,2011,False,credit_card,Computer,AZ,1985,2015,0,2016,False,False,False,False,C0Q1,C1Q1,C2Q2,C3Q3,C4Q2,C5Q0,C6Q7,C7Q9,C8Q0,C9Q7,C10Q7,C11Q0,C12Q3,C13Q0,C14Q0,C15Q0,C16Q6,C17Q8,C18Q0,C19Q0,C20Q0,C21Q2,C22Q3,C23Q2,C24Q3,C25Q0,C26Q0,C27Q0,C28Q1,C29Q7,C30Q7,C31Q0,C32Q0,__label__1
1,1,1077430,60months,C,C4,Ryder,1,RENT,Source Verified,2011,False,car,bike,GA,1999,2013,0,2016,False,False,False,False,C0Q0,C1Q0,C2Q0,C3Q7,C4Q0,C5Q0,C6Q3,C7Q0,C8Q0,C9Q7,C10Q7,C11Q3,C12Q3,C13Q0,C14Q0,C15Q0,C16Q1,C17Q1,C18Q0,C19Q0,C20Q0,C21Q0,C22Q0,C23Q0,C24Q1,C25Q0,C26Q1,C27Q1,C28Q1,C29Q0,C30Q0,C31Q0,C32Q0,__label__2


# Models

In [13]:
# Initialize lists to store evaluation metrics for each fold
dataset_used = []
model_used = []
data_balancing_technique = []
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
conf_matrices = []

In [14]:
#combined_metrics = pd.DataFrame()

combined_metrics = pd.DataFrame(columns=['dataset', 'model', 'data balancing technique', 'fold', 'precision_1','precision_2','recall_1','recall_2','f1-score_1','f1-score_2','support_1','support_2','TP','FP','TN','FN'])

# Fast Text 
### train_supervised

## DF8

In [25]:
# Assuming 'df2' is your DataFrame with features and the target column
# Replace 'features' with the actual list of feature columns
features = df8.drop(columns=['target'])  # Drop the target column to get the feature columns
target = df8['target'].astype(str)

# Define hyperparameters
dim = 300  # Dimension of word vectors
min_count = 1  # Minimum frequency of words
loss = 'ns'  # Hierarchical softmax loss #'hs', 'ns', 'softmax'
epoch = 100  # Number of training epochs
bucket = 2000000  # Number of buckets used for hashing n-grams
word_ngrams = 1  # Maximum length of word n-grams
lr = 0.5 #0.1, 0.01, 0.001 #learning rate

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Create a new column "content" by concatenating values from selected columns
    X_train['content'] = y_train + ' ' + X_train.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    X_test['content'] = X_test.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    
    X_train = X_train['content']
    X_test = X_test['content']
    
    X_train_list = [value for value in X_train]
    
    # Save training data to a file
    with open("train_german_fast_text.txt", "w", encoding='utf-8', errors='ignore') as f:
        for line in X_train_list:
            f.write(line + "\n")

    # Training data file path
    train_data_path = "train_german_fast_text.txt"
    
    # Train the supervised model
    model = fasttext.train_supervised(input=train_data_path, dim=dim, minCount=min_count,bucket=bucket, loss=loss, epoch=epoch, wordNgrams=word_ngrams)#lr=lr, 
                                       #lr=lr,bucket=bucket,
    
    # Predict on the testing set
    # Initialize an empty list to store the predicted labels
    y_pred_list = []

    # Iterate over each data point in X_test and predict the label
    for text in X_test:
        # Predict the label for the current text
        predicted_label, _ = model.predict(text)
        # Append the predicted label to the list
        y_pred_list.append(predicted_label[0])

    # Convert the list of predicted labels to a pandas Series
    y_pred = pd.Series(y_pred_list)
    
    # Extract the numeric label using a lambda function
    #y_test = y_test.apply(lambda x: int(x.split("__label__")[1]) if len(x.split("__label__")) > 1 else )
    y_test = y_test.apply(lambda x: int(x.split("__label__")[1]))  ## uncomment for next run
    y_pred = y_pred.apply(lambda x: int(x.split("__label__")[1]))
    
    # Model evaluation
    accuracy = accuracy_score(y_test, y_pred.round())
    precision = precision_score(y_test, y_pred.round(), average='weighted')
    recall = recall_score(y_test, y_pred.round(), average='weighted')
    f1 = f1_score(y_test, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred.round())
    
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF8',
    'model' : 'Fast Text - train_supervised',
    'data balancing technique' : 'None',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)
    
    print(i)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

2
3
4
5
6
7
8
9
10
11
Mean Accuracy: 0.99
Mean Precision: 0.99
Mean Recall: 0.99
Mean F1-Score: 0.99


In [34]:
combined_metrics[(combined_metrics['data balancing technique'] == 'None') & (combined_metrics['dataset'] == 'DF8')].head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
0,DF8,Fast Text - train_supervised,None,1,0.990191,0.984589,0.997530,0.941080,0.993847,0.962343,3643.0,611.0,3634,36,575,9
1,DF8,Fast Text - train_supervised,None,2,0.992341,0.993311,0.998899,0.954984,0.995609,0.973770,3632.0,622.0,3628,28,594,4


### FastText train_supervised with df4 data and imbalance data tackling (RandomUnderSampler)

In [28]:
# Assuming 'df8' is your DataFrame with features and the target column
# Replace 'features' with the actual list of feature columns
features = df8.drop(columns=['target'])  # Drop the target column to get the feature columns
target = df8['target'].astype(str)

# Define hyperparameters
dim = 300  # Dimension of word vectors
min_count = 1  # Minimum frequency of words
loss = 'ns'  # Hierarchical softmax loss #'hs', 'ns', 'softmax'
epoch = 100  # Number of training epochs
bucket = 2000000  # Number of buckets used for hashing n-grams
word_ngrams = 1  # Maximum length of word n-grams
lr = 0.5 #0.1, 0.01, 0.001 #learning rate

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Oversampling the minority class using SMOTE
    rus = RandomUnderSampler(random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
    
    # Create a new column "content" by concatenating values from selected columns
    X_train_resampled['content'] = y_train_resampled + ' ' + X_train_resampled.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    X_test['content'] = X_test.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    
    X_train = X_train_resampled['content']
    X_test = X_test['content']
    
    X_train_list = [value for value in X_train]
    
    # Save training data to a file
    with open("train_german_fast_text.txt", "w", encoding='utf-8', errors='ignore') as f:
        for line in X_train_list:
            f.write(line + "\n")

    # Training data file path
    train_data_path = "train_german_fast_text.txt"
    
    # Train the supervised model
    model = fasttext.train_supervised(input=train_data_path, dim=dim, minCount=min_count,bucket=bucket, loss=loss, epoch=epoch, wordNgrams=word_ngrams)#lr=lr, 
                                       #lr=lr,bucket=bucket,
    
    # Predict on the testing set
    # Initialize an empty list to store the predicted labels
    y_pred_list = []

    # Iterate over each data point in X_test and predict the label
    for text in X_test:
        # Predict the label for the current text
        predicted_label, _ = model.predict(text)
        # Append the predicted label to the list
        y_pred_list.append(predicted_label[0])

    # Convert the list of predicted labels to a pandas Series
    y_pred = pd.Series(y_pred_list)
    
    # Extract the numeric label using a lambda function
    y_test = y_test.apply(lambda x: int(x.split("__label__")[1]))  ## uncomment for next run
    y_pred = y_pred.apply(lambda x: int(x.split("__label__")[1]))
    
    # Model evaluation
    accuracy = accuracy_score(y_test, y_pred.round())
    precision = precision_score(y_test, y_pred.round(), average='weighted')
    recall = recall_score(y_test, y_pred.round(), average='weighted')
    f1 = f1_score(y_test, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred.round())
    
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF8',
    'model' : 'Fast Text - train_supervised',
    'data balancing technique' : 'Random Under Sampler',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)
    
    print(i)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

2
3
4
5
6
7
8
9
10
11
Mean Accuracy: 0.99
Mean Precision: 0.99
Mean Recall: 0.99
Mean F1-Score: 0.99


In [33]:
combined_metrics[(combined_metrics['data balancing technique'] == 'Random Under Sampler') & (combined_metrics['dataset'] == 'DF8')]#.head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
10,DF8,Fast Text - train_supervised,Random Under Sampler,1,0.994455,0.913447,0.984628,0.967267,0.989517,0.939587,3643.0,611.0,3587,20,591,56
11,DF8,Fast Text - train_supervised,Random Under Sampler,2,0.995834,0.929556,0.987335,0.975884,0.991566,0.952157,3632.0,622.0,3586,15,607,46
12,DF8,Fast Text - train_supervised,Random Under Sampler,3,0.995822,0.941265,0.989209,0.976562,0.992504,0.958589,3614.0,640.0,3575,15,625,39
13,DF8,Fast Text - train_supervised,Random Under Sampler,4,0.993863,0.929746,0.986981,0.965839,0.990410,0.947449,3610.0,644.0,3563,22,622,47
14,DF8,Fast Text - train_supervised,Random Under Sampler,5,0.996350,0.907514,0.982286,0.979719,0.989268,0.942236,3613.0,641.0,3549,13,628,64
15,DF8,Fast Text - train_supervised,Random Under Sampler,6,0.995784,0.899281,0.980626,0.976562,0.988147,0.936330,3613.0,640.0,3543,15,625,70
16,DF8,Fast Text - train_supervised,Random Under Sampler,7,0.995200,0.904360,0.981074,0.974242,0.988087,0.938001,3593.0,660.0,3525,17,643,68
17,DF8,Fast Text - train_supervised,Random Under Sampler,8,0.994091,0.928469,0.986045,0.968657,0.990052,0.948137,3583.0,670.0,3533,21,649,50
18,DF8,Fast Text - train_supervised,Random Under Sampler,9,0.993548,0.928779,0.986355,0.965257,0.989939,0.946667,3591.0,662.0,3542,23,639,49
19,DF8,Fast Text - train_supervised,Random Under Sampler,10,0.997483,0.933628,0.987538,0.985981,0.992485,0.959091,3611.0,642.0,3566,9,633,45


### FastText train_supervised with df2 data and imbalance data tackling (RandomOverSampler)

In [29]:
# Assuming 'df8' is your DataFrame with features and the target column
# Replace 'features' with the actual list of feature columns
features = df8.drop(columns=['target'])  # Drop the target column to get the feature columns
target = df8['target'].astype(str)

# Define hyperparameters
dim = 300  # Dimension of word vectors
min_count = 1  # Minimum frequency of words
loss = 'ns'  # Hierarchical softmax loss #'hs', 'ns', 'softmax'
epoch = 100  # Number of training epochs
bucket = 2000000  # Number of buckets used for hashing n-grams
word_ngrams = 1  # Maximum length of word n-grams
lr = 0.5 #0.1, 0.01, 0.001 #learning rate

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Oversampling the minority class using SMOTE
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    
    # Create a new column "content" by concatenating values from selected columns
    X_train_resampled['content'] = y_train_resampled + ' ' + X_train_resampled.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    X_test['content'] = X_test.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    
    X_train = X_train_resampled['content']
    X_test = X_test['content']
    
    X_train_list = [value for value in X_train]
    
    # Save training data to a file
    with open("train_german_fast_text.txt", "w", encoding='utf-8', errors='ignore') as f:
        for line in X_train_list:
            f.write(line + "\n")

    # Training data file path
    train_data_path = "train_german_fast_text.txt"
    
    # Train the supervised model
    model = fasttext.train_supervised(input=train_data_path, dim=dim, minCount=min_count,bucket=bucket, loss=loss, epoch=epoch, wordNgrams=word_ngrams)#lr=lr, 
                                       #lr=lr,bucket=bucket,
    
    # Predict on the testing set
    # Initialize an empty list to store the predicted labels
    y_pred_list = []

    # Iterate over each data point in X_test and predict the label
    for text in X_test:
        # Predict the label for the current text
        predicted_label, _ = model.predict(text)
        # Append the predicted label to the list
        y_pred_list.append(predicted_label[0])

    # Convert the list of predicted labels to a pandas Series
    y_pred = pd.Series(y_pred_list)
    
    # Extract the numeric label using a lambda function
    y_test = y_test.apply(lambda x: int(x.split("__label__")[1]))  ## uncomment for next run
    y_pred = y_pred.apply(lambda x: int(x.split("__label__")[1]))
    
    # Model evaluation
    accuracy = accuracy_score(y_test, y_pred.round())
    precision = precision_score(y_test, y_pred.round(), average='weighted')
    recall = recall_score(y_test, y_pred.round(), average='weighted')
    f1 = f1_score(y_test, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred.round())
    
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF8',
    'model' : 'Fast Text - train_supervised',
    'data balancing technique' : 'Random Over Sampler',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)
    
    print(i)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

2
3
4
5
6
7
8
9
10
11
Mean Accuracy: 0.99
Mean Precision: 0.99
Mean Recall: 0.99
Mean F1-Score: 0.99


In [32]:
combined_metrics[(combined_metrics['data balancing technique'] == 'Random Over Sampler') & (combined_metrics['dataset'] == 'DF8')].head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
20,DF8,Fast Text - train_supervised,Random Over Sampler,1,0.990461,0.984615,0.997530,0.942717,0.993982,0.963211,3643.0,611.0,3634,35,576,9
21,DF8,Fast Text - train_supervised,Random Over Sampler,2,0.992879,0.988391,0.998073,0.958199,0.995469,0.973061,3632.0,622.0,3625,26,596,7


## Performance metrics for the Linear Regression

In [35]:
combined_metrics

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
0,DF8,Fast Text - train_supervised,None,1,0.990191,0.984589,0.997530,0.941080,0.993847,0.962343,3643.0,611.0,3634,36,575,9
1,DF8,Fast Text - train_supervised,None,2,0.992341,0.993311,0.998899,0.954984,0.995609,0.973770,3632.0,622.0,3628,28,594,4
2,DF8,Fast Text - train_supervised,None,3,0.993396,0.993548,0.998893,0.962500,0.996137,0.977778,3614.0,640.0,3610,24,616,4
3,DF8,Fast Text - train_supervised,None,4,0.991738,0.985554,0.997507,0.953416,0.994614,0.969219,3610.0,644.0,3601,30,614,9
4,DF8,Fast Text - train_supervised,None,5,0.993119,0.991948,0.998616,0.960998,0.995860,0.976228,3613.0,641.0,3608,25,616,5
5,DF8,Fast Text - train_supervised,None,6,0.993111,0.985577,0.997509,0.960938,0.995305,0.973101,3613.0,640.0,3604,25,615,9
6,DF8,Fast Text - train_supervised,None,7,0.991985,0.993701,0.998887,0.956061,0.995424,0.974517,3593.0,660.0,3589,29,631,4
7,DF8,Fast Text - train_supervised,None,8,0.991133,0.990683,0.998325,0.952239,0.994716,0.971081,3583.0,670.0,3577,32,638,6
8,DF8,Fast Text - train_supervised,None,9,0.989244,0.993620,0.998886,0.941088,0.994042,0.966641,3591.0,662.0,3587,39,623,4
9,DF8,Fast Text - train_supervised,None,10,0.994483,0.990446,0.998338,0.968847,0.996407,0.979528,3611.0,642.0,3605,20,622,6


In [36]:
# Assuming df is your DataFrame
combined_metrics.to_csv('Output Data/Fast Text - train_supervised2.csv')